In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import json

In [ ]:
#建立头,用于模拟用户操作,防止反爬虫
headers = {
        'Authority':'www.mafengwo.cn',
        'Accept': 'application/json, text/javascript, */*',
        'Accept-Encoding': 'gzip, deflate',
        'Accept-Language': 'zh-CN,zh;q=0.9',
        'Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.77 Safari/537.36'
        }

In [ ]:
#此方法用于爬取城市美食排行,里面有两个参数, city代表爬取的城市;city_id代表城市对应的id
def Crawl_Data_Out_Food(city, city_id):
    rankUrl = 'http://www.mafengwo.cn/cy/{}/gonglve.html'.format(city_id)    #这里的网址{}里面的就是放置城市id的,你可以看一下其他城市美食的网页地址,区别就是id号不同
    res = requests.get(rankUrl,headers=headers)                              #例牌请求,beautifulSoup获取html
    soup_bs = BeautifulSoup(res.text)

    ary_rank = []                                          #新建数组用于保存每个美食的字典
    list_rank = soup_bs.find('ol',class_='list-rank')      #爬取美食名称,美食关注,美食排名,保存入字典dic_rank,加入数组ary_rank
    for rank in list_rank.find_all('li'):
        try:
            dic_rank = {}
            dic_rank['city'] = city
            dic_rank['rank'] = rank.find('span',class_='r-img').text
            dic_rank['cate'] = rank.find('h3').text
            dic_rank['poll'] = rank.find('span',class_='trend').text
            ary_rank.append(dic_rank)
        except:
            pass
    return ary_rank

In [ ]:
#此方法用于爬取第一页的城市名称和城市id
def Crawl_Data_Out_City():
    cityUrl = 'http://www.mafengwo.cn/mdd/citylist/21536.html'            #网址url为热门城市的网页
    res = requests.get(cityUrl,headers=headers)                           #例牌请求,beautifulSoup获取html
    soup_bs = BeautifulSoup(res.text)
    ary_city = []                                        #新建数组用于保存每个城市的字典
    list_city = soup_bs.find('ul',id='citylistlist')     #爬取城市名称和城市id
    for city in list_city.find_all('li',class_='item'):
        dic_city = {}
        str_city = city.find('div',class_='title').text.strip().replace(' ','').replace('\n','')    #删除城市前后的空格,并置换\n为空
        title_city = re.sub("[A-Za-z0-9\!\%\[\]\,\。]", "", str_city)     #删除城市名里面的数字,英文,符号
        dic_city['title'] = title_city
        dic_city['id'] = city.find('a').get('data-id')  #提取标签里的data-id属性
        ary_city.append(dic_city)
    return ary_city

In [ ]:
#此方法用于爬取第二页以后的城市名称和id,因为此网页是用post请求的,网址不变,点击第二页会发送一条post的请求
def Crawl_Data_Out_Other_City(page):
    data = {'mddid':'21536','page':''}                                   #这是post请求,所要携带的数据,可以在F12内找到
    data['page'] = str(page)                                             #page参数,数字page转换成字符并设置data字典,这里的page是指第几页
    url = 'http://www.mafengwo.cn/mdd/base/list/pagedata_citylist'       #可以在F12内可以看到此为post请求,请求url
    res = requests.post(url, data=data, headers=headers)                 #发送post请求,带上需要发的数据data
    json_html = json.loads(res.text)                                     #获取回来的是个json文件,解析json
    html = json_html['list']
    soup_bs = BeautifulSoup(html)                                 #后面爬取的,和第一页的类似
    ary_city = []
    for city in soup_bs.find_all('li',class_='item'):
        dic_city = {}
        str_city = city.find('div',class_='title').text.strip().replace(' ','').replace('\n','')
        title_city = re.sub("[A-Za-z0-9\!\%\[\]\,\。]", "", str_city)
        dic_city['title'] = title_city
        dic_city['id'] = city.find('a').get('data-id')
        ary_city.append(dic_city)
    return ary_city

In [ ]:
ary_city_rank = []           #新建城市美食数组
ary_citys = []               #新建城市数组
ary_citys.extend(Crawl_Data_Out_City())    #爬取第一页城市信息(名称/id),并加入数组
for i in range(2,6):                       #循环爬取第2至第5页的城市信息,可以修改(2,6)中的6来设置需要爬取多少页
    ary_citys.extend(Crawl_Data_Out_Other_City(i))     #爬取第2页至第n页的城市信息,这里i指页数
for city in ary_citys:                     #根据城市名称和id,循环爬取美食排行
    ary_city_rank.extend(Crawl_Data_Out_Food(city['title'],city['id']))

In [ ]:
ary_citys

In [27]:
ary_city_rank

[{'city': '香港', 'rank': '1', 'cate': '云吞', 'poll': '812'},
 {'city': '香港', 'rank': '2', 'cate': '蛋挞', 'poll': '590'},
 {'city': '香港', 'rank': '3', 'cate': '菠萝油', 'poll': '417'},
 {'city': '香港', 'rank': '4', 'cate': '鸳鸯奶茶', 'poll': '374'},
 {'city': '香港', 'rank': '5', 'cate': '烧腊', 'poll': '178'},
 {'city': '香港', 'rank': '6', 'cate': '车仔面', 'poll': '175'},
 {'city': '香港', 'rank': '7', 'cate': '杨枝甘露', 'poll': '161'},
 {'city': '香港', 'rank': '8', 'cate': '鱼蛋粉', 'poll': '158'},
 {'city': '香港', 'rank': '9', 'cate': '煎酿三宝', 'poll': '29'},
 {'city': '香港', 'rank': '10', 'cate': '牛腩粉', 'poll': '26'},
 {'city': '北京', 'rank': '1', 'cate': '北京烤鸭', 'poll': '1891'},
 {'city': '北京', 'rank': '2', 'cate': '炸酱面', 'poll': '1278'},
 {'city': '北京', 'rank': '3', 'cate': '瓷瓶儿酸奶', 'poll': '1256'},
 {'city': '北京', 'rank': '4', 'cate': '炒肝', 'poll': '859'},
 {'city': '北京', 'rank': '5', 'cate': '爆肚', 'poll': '837'},
 {'city': '北京', 'rank': '6', 'cate': '豆汁儿焦圈', 'poll': '809'},
 {'city': '北京', 'rank': '7', 'cate'

In [32]:
total_info = []
for i in range(0,len(ary_city_rank)):
    single_info = ary_city_rank[i].values()
    total_info.append(single_info)

In [33]:
total_info

[dict_values(['香港', '1', '云吞', '812']),
 dict_values(['香港', '2', '蛋挞', '590']),
 dict_values(['香港', '3', '菠萝油', '417']),
 dict_values(['香港', '4', '鸳鸯奶茶', '374']),
 dict_values(['香港', '5', '烧腊', '178']),
 dict_values(['香港', '6', '车仔面', '175']),
 dict_values(['香港', '7', '杨枝甘露', '161']),
 dict_values(['香港', '8', '鱼蛋粉', '158']),
 dict_values(['香港', '9', '煎酿三宝', '29']),
 dict_values(['香港', '10', '牛腩粉', '26']),
 dict_values(['北京', '1', '北京烤鸭', '1891']),
 dict_values(['北京', '2', '炸酱面', '1278']),
 dict_values(['北京', '3', '瓷瓶儿酸奶', '1256']),
 dict_values(['北京', '4', '炒肝', '859']),
 dict_values(['北京', '5', '爆肚', '837']),
 dict_values(['北京', '6', '豆汁儿焦圈', '809']),
 dict_values(['北京', '7', '卤煮', '575']),
 dict_values(['北京', '8', '豌豆黄', '453']),
 dict_values(['北京', '9', '驴打滚', '446']),
 dict_values(['北京', '10', '冰糖葫芦', '376']),
 dict_values(['厦门', '1', '沙茶面', '3347']),
 dict_values(['厦门', '2', '海蛎煎', '2396']),
 dict_values(['厦门', '3', '花生汤', '1841']),
 dict_values(['厦门', '4', '土笋冻', '1825']),
 dict_

In [34]:
import csv
with open('chongqing_food.csv','w',encoding='utf-8-sig') as f:
    writer = csv.writer(f)
    header = ['city','rank', 'cate', 'poll']
    writer.writerow(header)
    writer.writerows(total_info)

In [2]:
import pandas as pd
df = pd.read_csv('chongqing_food.csv')

In [3]:
df

,city,rank,cate,poll
0,香港,1,云吞,812
1,香港,2,蛋挞,590
2,香港,3,菠萝油,417
3,香港,4,鸳鸯奶茶,374
4,香港,5,烧腊,178
5,香港,6,车仔面,175
6,香港,7,杨枝甘露,161
7,香港,8,鱼蛋粉,158
8,香港,9,煎酿三宝,29
9,香港,10,牛腩粉,26


In [4]:
df.sort_values(by='poll',ascending=False)

,city,rank,cate,poll
20,厦门,1,沙茶面,3347
74,重庆,1,火锅,2916
21,厦门,2,海蛎煎,2396
10,北京,1,北京烤鸭,1891
22,厦门,3,花生汤,1841
23,厦门,4,土笋冻,1825
56,丽江,1,腊排骨,1659
64,西安',1,肉夹馍,1656
275,鼓浪屿,1,海鲜,1589
160,桂林,1,漓江鱼,1581
